In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from flaml.autogen.agentchat2.llm_chat import (
    LLMChatMessage,
    LLMChatAgent,
    LLMChatMessageStream,
    llm_chat_action,
    llm_chat_trigger,
)


In [3]:
stream = LLMChatMessageStream()

In [4]:
participant1 = LLMChatAgent(
    name="participant1",
    message_stream=stream,
    partner_name="participant2",
    system_message={
        "role": "system",
        "content": "You are debating with user whether it is better to travel to Yellow Stone from Seattle by car or by flight. Your position is that car is better.",
    },
    llm_config={"temperature": 0.9, "model": "gpt-3.5-turbo"},
)
participant2 = LLMChatAgent(
    name="participant2",
    message_stream=stream,
    partner_name="participant1",
    system_message={
        "role": "system",
        "content": "You are debating with user whether it is better to travel to Yellow Stone from Seattle by car or by flight. Your position is that flight is better.",
    },
    llm_config={"temperature": 0.9, "model": "gpt-3.5-turbo"},
)

2023-08-04 11:16:03,167 DEBUG Adding subscriber <flaml.autogen.agentchat2.llm_chat.LLMChatAgent object at 0x10d732950> to address participant1
2023-08-04 11:16:03,167 DEBUG Adding subscriber <flaml.autogen.agentchat2.llm_chat.LLMChatAgent object at 0x10647b310> to address participant2


In [5]:
participant1.register(lambda msg, ctx: llm_chat_trigger(msg, ctx) and len(ctx.chat_history) < 10, llm_chat_action)
participant2.register(lambda msg, ctx: llm_chat_trigger(msg, ctx) and len(ctx.chat_history) < 10, llm_chat_action)

In [6]:
stream.send(
    LLMChatMessage(
        sender_name="participant2",
        receiver_name="participant1",
        message={"role": "user", "content": "It is better to travel to Yellow Stone by flight."},
    ),
)

2023-08-04 11:16:03,194 DEBUG Send message LLMChatMessage(sender_name=participant2, receiver_name=participant1, message={'role': 'user', 'content': 'It is better to travel to Yellow Stone by flight.'}) to address participant1


In [7]:
stream.run()

2023-08-04 11:16:20,527 DEBUG Send message LLMChatMessage(sender_name=participant1, receiver_name=participant2, message={'role': 'user', 'content': "I understand that flying might seem like a convenient option, but I believe that traveling to Yellowstone by car is a better choice. Here are a few reasons why:\n\n1. Flexibility: When you travel by car, you have the freedom to set your own schedule and make spontaneous stops along the way. You can explore other beautiful areas or attractions that you may come across during your journey. This flexibility allows you to make the most out of your trip and have a more personalized experience.\n\n2. Scenic Route: The drive from Seattle to Yellowstone is incredibly scenic, with stunning landscapes and natural beauty to admire along the way. You can enjoy breathtaking views of mountains, forests, and lakes as you drive through states like Washington, Idaho, and Montana. This journey can be an adventure in itself, offering memorable moments that y